In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

# 데이터를 불러온다.
trn = pd.read_csv('C:\\Users\\Choi\\Desktop\\kaggle\\train_ver2.csv')
tst = pd.read_csv('C:\\Users\\Choi\\Desktop\\kaggle\\test_ver2.csv')

C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
## 데이터 전처리 ##

# 제품 변수를 별도로 저장해 놓는다.
prods = trn.columns[24:].tolist()

# 제품 변수 결측값을 미리 0으로 대체한다.
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터를 제거한다.
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

# 훈련 데이터와 테스트 데이터를 통합한다. 테스트 데이터에 없는 제품 변수는 0으로 채운다.
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# 학습에 사용할 변수를 담는 list이다.
features = []

# 범주형 변수를 .factorize() 함수를 통해 label encoding한다.
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환한다.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# 학습에 사용할 수치형 변수를 features에 추구한다.
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

In [9]:
# (피쳐 엔지니어링) 두 날짜 변수에서 연도와 월 정보를 추출한다.
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

# 그 외 변수의 결측값은 모두 -99로 대체한다.
df.fillna(-99, inplace=True)

# (피쳐 엔지니어링) lag-1 데이터를 생성한다.
# 코드 2-12와 유사한 코드 흐름이다.

# 날짜를 숫자로 변환하는 함수이다. 2015-01-28은 1, 2016-06-28은 18로 변환된다
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. Lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다
del df, df_lag

# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

# lag-1 변수를 추가한다.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

In [10]:
###
### Baseline 모델 이후, 다양한 피쳐 엔지니어링을 여기에 추가한다.
###


## 모델 학습
# 학습을 위하여 데이터를 훈련, 테스트용으로 분리한다.
# 학습에는 2016-01-28 ~ 2016-04-28 데이터만 사용하고, 검증에는 2016-05-28 데이터를 사용한다.
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

In [13]:
# 훈련 데이터에서 신규 구매 건수만 추출한다.
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

# 훈련, 검증 데이터로 분리한다. 
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

In [14]:
# XGBoost 모델 parameter를 설정한다.
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

# 훈련, 검증 데이터를 XGBoost 형태로 변환한다.
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

# XGBoost 모델을 훈련 데이터로 학습한다!
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[0]	train-mlogloss:2.72501	eval-mlogloss:2.73552
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:2.45696	eval-mlogloss:2.47073
[2]	train-mlogloss:2.27435	eval-mlogloss:2.28931
[3]	train-mlogloss:2.13565	eval-mlogloss:2.15204
[4]	train-mlogloss:2.02201	eval-mlogloss:2.03829
[5]	train-mlogloss:1.92567	eval-mlogloss:1.94297
[6]	train-mlogloss:1.84739	eval-mlogloss:1.86529
[7]	train-mlogloss:1.77947	eval-mlogloss:1.79788
[8]	train-mlogloss:1.71764	eval-mlogloss:1.73641
[9]	train-mlogloss:1.66492	eval-mlogloss:1.68415
[10]	train-mlogloss:1.61812	eval-mlogloss:1.63771
[11]	train-mlogloss:1.57482	eval-mlogloss:1.59515
[12]	train-mlogloss:1.53556	eval-mlogloss:1.55617
[13]	train-mlogloss:1.50161	eval-mlogloss:1.52287
[14]	train-mlogloss:1.47083	eval-mlogloss:1.49261
[15]	train-mlogloss:1.44125	eval-mlogloss:1.46339
[16]	train-mlogloss:1.41514	eval-mlogloss:1.43768
[17]	trai

[161]	train-mlogloss:0.999427	eval-mlogloss:1.0873
[162]	train-mlogloss:0.998843	eval-mlogloss:1.08729
[163]	train-mlogloss:0.998339	eval-mlogloss:1.08726
[164]	train-mlogloss:0.997724	eval-mlogloss:1.08724
[165]	train-mlogloss:0.997298	eval-mlogloss:1.08726
[166]	train-mlogloss:0.99679	eval-mlogloss:1.08722
[167]	train-mlogloss:0.996205	eval-mlogloss:1.08712
[168]	train-mlogloss:0.995684	eval-mlogloss:1.08709
[169]	train-mlogloss:0.995162	eval-mlogloss:1.08708
[170]	train-mlogloss:0.994694	eval-mlogloss:1.08705
[171]	train-mlogloss:0.993981	eval-mlogloss:1.08699
[172]	train-mlogloss:0.993395	eval-mlogloss:1.08694
[173]	train-mlogloss:0.992896	eval-mlogloss:1.08693
[174]	train-mlogloss:0.992331	eval-mlogloss:1.08693
[175]	train-mlogloss:0.991863	eval-mlogloss:1.08691
[176]	train-mlogloss:0.991429	eval-mlogloss:1.08693
[177]	train-mlogloss:0.990817	eval-mlogloss:1.08695
[178]	train-mlogloss:0.990209	eval-mlogloss:1.08696
[179]	train-mlogloss:0.98966	eval-mlogloss:1.08696
[180]	train-mlo

In [15]:
# 학습한 모델을 저장한다.
import pickle
pickle.dump(model, open("../model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit

In [27]:
# MAP@7 평가 척도를 위한 준비작업이다.
from mapk import mapk
# 고객 식별 번호를 추출한다.
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])
# 검증 데이터에서 신규 구매를 구한다.
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]    
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

# 고객별 신규 구매 정답 값을 add_vld_list에 저장하고, 총 count를 count_vld에 저장한다.
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

# 검증 데이터에서 얻을 수 있는 MAP@7 최고점을 미리 구한다. (0.042663)
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

# 검증 데이터에 대한 예측 값을 구한다.
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

# 저번 달에 보유한 제품은 신규 구매가 불가하기 때문에, 확률값에서 미리 1을 빼준다
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])

# 검증 데이터 예측 상위 7개를 추출한다.
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])
    
# 검증 데이터에서의 MAP@7 점수를 구한다. (0.036466)
print(mapk(add_vld_list, result_vld, 7, 0.0))

C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


0.04266379915553903


C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


0.03646758854670042


In [28]:
# XGBoost 모델을 전체 훈련 데이터로 재학습한다!
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]
# 트리 개수를 늘어난 데이터 양만큼 비례해서 증가한다.
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
# XGBoost 모델 재학습!
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


[0]	train-mlogloss:2.72659
[1]	train-mlogloss:2.4579
[2]	train-mlogloss:2.27488
[3]	train-mlogloss:2.1356
[4]	train-mlogloss:2.02158
[5]	train-mlogloss:1.9252
[6]	train-mlogloss:1.84685
[7]	train-mlogloss:1.77856
[8]	train-mlogloss:1.71673
[9]	train-mlogloss:1.66394
[10]	train-mlogloss:1.61715
[11]	train-mlogloss:1.57391
[12]	train-mlogloss:1.53473
[13]	train-mlogloss:1.50084
[14]	train-mlogloss:1.47
[15]	train-mlogloss:1.44046
[16]	train-mlogloss:1.41425
[17]	train-mlogloss:1.39089
[18]	train-mlogloss:1.36909
[19]	train-mlogloss:1.34917
[20]	train-mlogloss:1.33089
[21]	train-mlogloss:1.31322
[22]	train-mlogloss:1.29747
[23]	train-mlogloss:1.28292
[24]	train-mlogloss:1.26957
[25]	train-mlogloss:1.25662
[26]	train-mlogloss:1.24471
[27]	train-mlogloss:1.23375
[28]	train-mlogloss:1.22354
[29]	train-mlogloss:1.21445
[30]	train-mlogloss:1.20597
[31]	train-mlogloss:1.19733
[32]	train-mlogloss:1.18932
[33]	train-mlogloss:1.18199
[34]	train-mlogloss:1.175
[35]	train-mlogloss:1.16834
[36]	train

In [29]:
# 변수 중요도를 출력해본다. 예상하던 변수가 상위로 올라와 있는가?
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

Feature importance:
('renta', 20865)
('age', 20122)
('antiguedad', 18398)
('age_prev', 13287)
('antiguedad_prev', 13128)
('fecha_alta_month', 12248)
('nomprov', 11603)
('fecha_alta_year', 9743)
('renta_prev', 8897)
('canal_entrada', 7840)
('nomprov_prev', 6753)
('canal_entrada_prev', 4824)
('fecha_alta_month_prev', 4439)
('ind_recibo_ult1_prev', 3467)
('sexo', 3383)
('fecha_alta_year_prev', 3378)
('ind_ecue_fin_ult1_prev', 3099)
('ind_cco_fin_ult1_prev', 3060)
('ind_cno_fin_ult1_prev', 2943)
('segmento', 2253)
('ind_tjcr_fin_ult1_prev', 2174)
('ind_reca_fin_ult1_prev', 2151)
('segmento_prev', 1964)
('tiprel_1mes', 1723)
('ind_nom_pens_ult1_prev', 1683)
('ind_valo_fin_ult1_prev', 1646)
('ind_ctop_fin_ult1_prev', 1574)
('ind_dela_fin_ult1_prev', 1561)
('ind_nomina_ult1_prev', 1504)
('sexo_prev', 1341)
('ind_actividad_cliente', 1340)
('ind_ctpp_fin_ult1_prev', 1165)
('tiprel_1mes_prev', 1152)
('ind_fond_fin_ult1_prev', 1035)
('ind_ctma_fin_ult1_prev', 912)
('ind_actividad_cliente_prev', 9

In [30]:
# 캐글 제출을 위하여 테스트 데이터에 대한 예측 값을 구한다.
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods])

C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Users\Choi\Anaconda3\envs\kaggle_env\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [31]:
# 제출 파일을 생성한다.
submit_file = open('../model/xgb.baseline.2015-06-28', 'w')
submit_file.write('ncodpers,added_products\n')
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))